# Kinesis Data Stream
* https://github.com/aws-samples/aws-ml-data-lake-workshop
* https://aws.amazon.com/blogs/big-data/snakes-in-the-stream-feeding-and-eating-amazon-kinesis-streams-with-python/

In [ ]:
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)
kinesis = boto3.Session().client(service_name='kinesis', region_name=region)
sts = boto3.Session().client(service_name='sts', region_name=region)

# Create a Kinesis Data Stream

![Kinesis Data Stream](img/kinesis_data_stream_docs.png)

In [ ]:
stream_name = "dsoaws-data-stream"
shard_count = 2

In [ ]:
response = kinesis.create_stream(
    StreamName=stream_name, 
    ShardCount=shard_count
)

In [ ]:
import time

status = ''
while status != 'ACTIVE':    
    r = kinesis.describe_stream(StreamName=stream_name)
    description = r.get('StreamDescription')
    status = description.get('StreamStatus')
    time.sleep(5)
    
print('Stream {} is active'.format(stream_name))

In [ ]:
print(response)

In [ ]:
data_stream_response = kinesis.describe_stream(
    StreamName=stream_name
)

print(data_stream_response)

In [ ]:
data_stream_arn = data_stream_response['StreamDescription']['StreamARN']
print(data_stream_arn)

In [ ]:
iam_kinesis_role_name = 'DSOAWS_Kinesis'

In [ ]:
assume_role_policy_doc = {
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Principal": {
        "Service": "kinesis.amazonaws.com"
      },
      "Action": "sts:AssumeRole"
    },
    {
      "Effect": "Allow",
      "Principal": {
        "Service": "firehose.amazonaws.com"
      },
      "Action": "sts:AssumeRole"
    }      
  ]
} 

In [ ]:
import json
import boto3
import time

from botocore.exceptions import ClientError

try:
    iam = boto3.client('iam')

    iam_role_kinesis = iam.create_role(
        RoleName=iam_kinesis_role_name,
        AssumeRolePolicyDocument=json.dumps(assume_role_policy_doc),
        Description='DSOAWS Kinesis Role'
    )
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        iam_role_kinesis = iam.get_role(RoleName=iam_kinesis_role_name)
        print("Role already exists")
    else:
        print("Unexpected error: %s" % e)
        
time.sleep(10)

In [ ]:
iam_role_kinesis_arn = iam_role_kinesis['Role']['Arn']
print(iam_role_kinesis_arn)

iam_role_kinesis_name = iam_role_kinesis['Role']['RoleName']
print(iam_role_kinesis_name)

In [ ]:
account_id = sts.get_caller_identity()['Account']

In [ ]:
kinesis_policy_doc = {
    "Version": "2012-10-17",
    "Statement": [
         {
            "Effect": "Allow",
            "Action": [
                "firehose:DeleteDeliveryStream",
                "firehose:PutRecord",
                "firehose:PutRecordBatch",
                "firehose:UpdateDestination",
            ],
            "Resource": [
                "arn:aws:firehose:{}:{}:deliverystream/{}".format(region, account_id, stream_name)
            ]
         },
         {
            "Effect": "Allow",
            "Action": [
                "kinesis:Get*",
                "kinesis:DescribeStream",
                "kinesis:Put*",
                "kinesis:List*",                
            ],
            "Resource": [
                "arn:aws:kinesis:{}:{}:stream/{}".format(region, account_id, stream_name)
            ]
         }
        
    ]
}

print(kinesis_policy_doc)

# Update Policy

In [ ]:
import time

response = iam.put_role_policy(
    RoleName=iam_role_kinesis_name,
    PolicyName='DSOAWS_KinesisPolicy',
    PolicyDocument=json.dumps(kinesis_policy_doc)
)

time.sleep(10)

In [ ]:
print(response)

# Create a Kinesis Firehose Stream with Source Data Stream

![](img/kinesis_firehose_s3_docs.png)

# Store Variables for the Next Notebooks

In [ ]:
%store stream_name

In [ ]:
%store data_stream_arn

In [ ]:
%store iam_role_kinesis_arn

In [ ]:
%%javascript
Jupyter.notebook.save_checkpoint();
Jupyter.notebook.session.delete();